In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics   

In [2]:
train = pd.read_csv('train_data.csv')

In [3]:
test = pd.read_csv('test_data.csv')
test = test.drop(test.columns[0], axis=1)

In [4]:
target = pd.read_csv('train_target.csv', header = None)
target.columns = ['pred']

In [5]:
train = train.drop('default', 1)

In [6]:
test = test.drop('default', 1)

In [7]:
cat_cols = []
num_cols = []
for col in train.columns:
    if len(train[col].unique()) <= 10:
        cat_cols.append(col)
    else:
        num_cols.append(col)    

In [8]:
cat_cols.append('job')
num_cols.remove('job')
num_cols.extend(['previous', 'emp.var.rate'])
cat_cols.remove('previous')
cat_cols.remove('emp.var.rate')

In [9]:
for feat in cat_cols:
    train[feat] = train[feat].astype('category')

In [10]:
train = pd.get_dummies(train)

In [11]:
for feat in cat_cols:
    test[feat] = test[feat].astype('category')

In [12]:
test = pd.get_dummies(test)

In [13]:
def modelfit(model, dtrain, target, predictors, useTrainCV=True):
    
    if useTrainCV:
        param = model.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain.values, label=target.values)
        cvresult = xgb.cv(param, xgtrain, num_boost_round = model.get_params()['n_estimators'], nfold = 5,
            metrics='auc', early_stopping_rounds = 50, show_progress=False)
        model.set_params(n_estimators=cvresult.shape[0])
    
    model.fit(dtrain, target, eval_metric='auc')
        
    predicts = model.predict(dtrain)
    predprob = model.predict_proba(dtrain)[:,1]

In [14]:
xgb3 = XGBClassifier( learning_rate =0.1, n_estimators=1000, max_depth=5, min_child_weight=3, gamma=0.20000000000000001,
                     subsample=0.9, colsample_bytree=0.9, reg_alpha=1e-05, objective= 'binary:logistic', nthread=4,
                     scale_pos_weight=1, seed=27)
modelfit(xgb3, train, target['pred'], train.columns)

Will train until cv error hasn't decreased in 50 rounds.
C:\Users\Victoria\Anaconda3\lib\site-packages\xgboost-0.4-py3.5.egg\xgboost\training.py:272: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  idset = [randidx[(i * kstep): min(len(randidx), (i + 1) * kstep)] for i in range(nfold)]
Stopping. Best iteration:
[137] cv-mean:0.9489478	cv-std:0.0021270612026925493


In [15]:
answer = pd.DataFrame({'Prediction': xgb3.predict_proba(test)[:,1]})

In [16]:
answer.to_csv('results1.csv', index_label = 'Id')